In [1]:
import yfinance as yf
import pandas as pd
import snowflake.connector
import os

In [2]:
# Connect snowflake to python
conn = snowflake.connector.connect(
    user = 'CHAITANYA35',
    password = '',
    account = '',
    warehouse = 'COMPUTE_WH',
    database = 'ISM',
    schema = 'DATA'
)

cursor = conn.cursor()

In [3]:
def get_stock_data(stock_symbol):
    
    stock_name = stock_symbol.upper() + ".NS"

    try:
        # create class for yf.
        stock_data = yf.Ticker(stock_name)

        # get all historical adjusted data for stock.
        all_data = stock_data.history(period="max", auto_adjust=True)

        # if stock returns no data flag error.
        if all_data.empty:
            print(f"No data found for {stock_name}")
            return pd.DataFrame()

        # reset index to get date as date column.
        all_data.reset_index(inplace= True)

        # rename all columns as per snowflake tables.
        all_data = all_data.rename(columns={
            'Date' : 'price_date',
            'Open' : 'open_price',
            'High': 'high_price',
            'Low': 'low_price',
            'Close': 'close_price',
            'Volume': 'volume'
        })

        # add stock_symbol column as well.
        all_data['stock_symbol'] = stock_symbol.upper()
        all_data['price_date'] = pd.to_datetime(all_data['price_date']).dt.strftime('%Y-%m-%d')

        # reorder all columns.
        all_data = all_data[['stock_symbol', 'price_date', 'open_price', 'high_price', 'low_price', 'close_price', 'volume']]

        return all_data
    
    except Exception as e:
        print(f"Error fetching data for {stock_name}")
        return pd.DataFrame()


In [4]:
# def save_and_upload():
#     # Truncate Table First.
#     cursor.execute("Truncate table stock_prices_single_load")

#     # saving dataframe to parquet file
#     parquet_file = 'temp_stock_data_single_load.parquet'
#     df_all_stocks_data.to_parquet(parquet_file, index=False, engine='pyarrow')

#     df_all_stocks_data['price_date'] = df_all_stocks_data['price_date'].astype(str)

#     # absolute_path = os.path.abspath(parquet_file)
#     absolute_path = "D://Data_Engineering//temp_stock_data_single_load.parquet"
#     basename = os.path.basename(parquet_file)

#     cursor.execute("CREATE STAGE IF NOT EXISTS stage_stock_prices_single_load")
#     cursor.execute(F"PUT file://{absolute_path} @stage_stock_prices_single_load OVERWRITE=True")

#     cursor.execute(f"""
#         COPY INTO stock_prices_single_load
#         FROM @stage_stock_prices_single_load/{basename}
#         FILE_FORMAT = (TYPE = 'PARQUET')
#         MATCH_BY_COLUMN_NAME=CASE_INSENSITIVE
#         """)
    

In [5]:
if __name__ == "__main__":
    cursor.execute("select * from stock_symbol")
    
    stock_list = [row[0] for row in cursor.fetchall()]

    df_all_stocks_data = pd.DataFrame()

    for stock in stock_list:
        df = get_stock_data(stock)
        if not df.empty:
            df_all_stocks_data = pd.concat([df_all_stocks_data,df], ignore_index=True)  

    # Check dataframe information
    print(df_all_stocks_data.count()) 

    # Truncate Table First.
    cursor.execute("Truncate table stock_prices_single_load")

    # saving dataframe to parquet file
    parquet_file = 'temp_stock_data_single_load.parquet'
    df_all_stocks_data.to_parquet(parquet_file, index=False, engine='pyarrow')

    df_all_stocks_data['price_date'] = df_all_stocks_data['price_date'].astype(str)

    # absolute_path = os.path.abspath(parquet_file)
    absolute_path = "D://Data_Engineering//temp_stock_data_single_load.parquet"
    basename = os.path.basename(parquet_file)

    cursor.execute("CREATE STAGE IF NOT EXISTS stage_stock_prices_single_load")
    cursor.execute(F"PUT file://{absolute_path} @stage_stock_prices_single_load OVERWRITE=True")

    cursor.execute(f"""
        COPY INTO stock_prices_single_load
        FROM @stage_stock_prices_single_load/{basename}
        FILE_FORMAT = (TYPE = 'PARQUET')
        MATCH_BY_COLUMN_NAME=CASE_INSENSITIVE
        """)
    
    print("All data loaded successfully.")
    
    # cursor.close()
    # conn.close()

stock_symbol    2279199
price_date      2279199
open_price      2279198
high_price      2279198
low_price       2279198
close_price     2279198
volume          2279199
dtype: int64
All data loaded successfully.


In [6]:
# # Truncate Table First.
# cursor.execute("Truncate table stock_prices")

# # saving dataframe to parquet file
# parquet_file = 'temp_df_all_stocks_data.parquet'
# df_all_stocks_data.to_parquet(parquet_file, index=False, engine='pyarrow')

# df_all_stocks_data['price_date'] = df_all_stocks_data['price_date'].astype(str)

# # absolute_path = os.path.abspath(parquet_file)
# absolute_path = "D://Data_Engineering//temp_df_all_stocks_data.parquet"
# basename = os.path.basename(parquet_file)

# cursor.execute("CREATE STAGE IF NOT EXISTS stage_stock_prices")
# cursor.execute(F"PUT file://{absolute_path} @stage_stock_prices OVERWRITE=True")

# cursor.execute(f"""
#     COPY INTO stock_prices
#     FROM @stage_stock_prices/{basename}
#     FILE_FORMAT = (TYPE = 'PARQUET')
#     MATCH_BY_COLUMN_NAME=CASE_INSENSITIVE
#     """)

# print("All data loaded successfully.")

# # cursor.close()
# # conn.close()